In [1]:
# 当y、yhat、ycomp是从所有单品中取值组成的序列时，当在第一大类中出现（y-ycomp==0），不能直接区分没有补偿和完全准确补偿的情况；
# 当在第二大类中出现（y-yhat==0），不能直接区分没有预测和完全准确预测的情况；
# 当在第三大类中出现（y-yhat==y-ycomp==0），不能直接区分没有预测、没有补偿、完全预测、完全补偿这四种情况；
# 基于上述原因，会导致无法准确统计某些分项的指标；建议y、yhat、ycomp从所有发生补偿的单品中取值，将没有发生补偿的单品排除。

# import random
# import pandas as pd
# import numpy as np

# weights = []
# for i in range(-99,1):
#     weights.append((0.94)**i)

# k1, k2, k3 = 5000, 4000, 1100
# y = pd.Series(random.choices(range(0,100), k=k1, weights=weights))  # 生成随机序列，模拟k1个单品在某一周的真实销量

# z1 = pd.Series(10*np.sin(random.choices(range(0,5000), k=k2)))  # 假定k1个单品中只有k2个有预测销量
# z2 = pd.Series([0 for _ in range(k1-k2)])
# z3 = pd.concat([z1, z2])
# z4 = z3.sample(frac=1, replace=True)  # 打乱series的顺序
# z4.reset_index(drop=True, inplace=True)
# yhat = y + z4

# z5 = pd.Series(10*np.sin(random.choices(range(0,5000), k=k3)))  # 假定k1个单品中只有k3个发生补偿
# z6 = pd.Series([0 for _ in range(k1-k3)])
# z7 = pd.concat([z1, z2])
# z8 = z7.sample(frac=1, replace=True)  # 打乱series的顺序
# z8.reset_index(drop=True, inplace=True)
# ycomp = y + z8

In [2]:
import random
import pandas as pd
import numpy as np

weights = []
for i in range(-99,1):
    weights.append((0.94)**i)

#  真实值组成的序列y、预测值组成的序列yhat、补偿值组成的序列ycomp，这三条序列组成中的单品是一一对应的，即同一单品在三条序列中的index相同
k1 = 5000
y = pd.Series(random.choices(range(0,100), k=k1, weights=weights))  # 生成随机序列，模拟5000个单品在某一周的真实销量

z1 = pd.Series(np.sin(random.choices(range(0,100), k=k1)))  # 假定这5000个是都有预测销量的单品
yhat = y + z1

z2 = pd.Series(np.sin(random.choices(range(0,100), k=k1)))  # 并且假定这5000个是都发生补偿的单品
ycomp = y + z2

In [3]:
# 得到经过第一层的三大类条件判断之后的真实值序列y_s1/2/3、预测值序列yhat_s1/2/3、补偿值序列ycomp_s1/2/3.

# 1. 补偿变好这一大类：
y_s1 = y[abs(y-yhat)>abs(y-ycomp)]  # 选出y序列中补偿变好的这一大类单品
yhat_s1 = yhat[abs(y-yhat)>abs(y-ycomp)]  # 选出yhat序列中补偿变好的这一大类单品
ycomp_s1 = ycomp[abs(y-yhat)>abs(y-ycomp)]  # 选出ycomp序列中补偿变好的这一大类单品

# 2. 补偿变差这一大类：
y_s2 = y[abs(y-yhat)<abs(y-ycomp)]  # 选出y序列中补偿变差的这一大类单品
yhat_s2 = yhat[abs(y-yhat)<abs(y-ycomp)]  # 选出yhat序列中补偿变差的这一大类单品
ycomp_s2 = ycomp[abs(y-yhat)<abs(y-ycomp)]  # 选出ycomp序列中补偿变差的这一大类单品

# 3. 补偿无效果这一大类：
y_s3 = y[abs(y-yhat)==abs(y-ycomp)]  # 选出y序列中补偿无效果的这一大类单品
yhat_s3 = yhat[abs(y-yhat)==abs(y-ycomp)]  # 选出yhat序列中补偿无效果的这一大类单品
ycomp_s3 = ycomp[abs(y-yhat)==abs(y-ycomp)]  # 选出ycomp序列中补偿无效果的这一大类单品

In [4]:
if len(y) == len(yhat) == len(ycomp):  # 若该条件满足，则必有：len(y_s1)==len(yhat_s1)==len(ycomp_s1), len(y_s2)==len(yhat_s2)==len(ycomp_s2), len(y_s3)==len(yhat_s3)==len(ycomp_s3).
    # 因为这里只需获取三大类情况各自的单品数量m1/m2/m3，所以从y、yhat、ycomp任一序列中选取单品，得到的m1/m2/m3都相等。下面是从y序列中选取单品，得到的yhat序列的长度m1/m2/m3.
    m1, m2, m3 = len(y_s1), len(y_s2), len(y_s3)
    mt = m1+m2+m3
    if mt == k1:
        print('计算正确，补偿变好、变坏、无效果三种情况的单品数相加与发生补偿的单品总数相等：\n')
        print('补偿变好的单品数有：{:.0f}个，占总单品比例：{:.2f}%'.format(m1, m1/mt*100))
        print('补偿变差的单品数有：{:.0f}个，占总单品比例：{:.2f}%'.format(m2, m2/mt*100))
        print('补偿无效果的单品数有：{:.0f}个，占总单品比例：{:.2f}%'.format(m3, m3/mt*100))
    else: print('计算有误，变好、变化、无效果三种情况的单品数相加与发生补偿的单品总数不等。')
else: print('真实值、预测值、补偿值组成的序列长度不等，建议化为等长，以避免后续某些指标代表的成分不唯一的情况出现。')
 

计算正确，补偿变好、变坏、无效果三种情况的单品数相加与发生补偿的单品总数相等：

补偿变好的单品数有：2497个，占总单品比例：49.94%
补偿变差的单品数有：2464个，占总单品比例：49.28%
补偿无效果的单品数有：39个，占总单品比例：0.78%


In [5]:
# 在经过第一层条件判断之后，得到经过第二层条件判断的各个真实值序列、预测值序列、补偿值序列


# 对于补偿变好这一大类中的6小类情况：

# 1.1 预测补偿均低，补偿同侧变好：
y_s1_c1 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
yhat_s1_c1 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
ycomp_s1_c1 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
# 1.2 预测补偿均高，补偿同侧变好：
y_s1_c2 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
yhat_s1_c2 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
ycomp_s1_c2 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
# 1.3 预低补高，补偿对侧变好：
y_s1_c3 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
yhat_s1_c3 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
ycomp_s1_c3 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
# 1.4 预高补低，补偿对侧变好：
y_s1_c4 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
yhat_s1_c4 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
ycomp_s1_c4 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
# 1.5 预低，完全补偿：
y_s1_c5 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
yhat_s1_c5 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
ycomp_s1_c5 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
# 1.6 预高，完全补偿：
y_s1_c6 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]
yhat_s1_c6 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]
ycomp_s1_c6 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]


# 对于补偿变差这一大类中的6小类情况：

# 2.1 预测补偿均低，补偿同侧变差：
y_s2_c1 = y_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
yhat_s2_c1 = yhat_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c1 = ycomp_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
# 2.2 预测补偿均高，补偿同侧变差：
y_s2_c2 = y_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
yhat_s2_c2 = yhat_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c2 = ycomp_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
# 2.3 预低补高，补偿对侧变差：
y_s2_c3 = y_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
yhat_s2_c3 = yhat_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c3 = ycomp_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
# 2.4 预高补低，补偿对侧变差：
y_s2_c4 = y_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
yhat_s2_c4 = yhat_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c4 = ycomp_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
# 2.5 完全准确预测，补偿偏低：
y_s2_c5 = y_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
yhat_s2_c5 = yhat_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c5 = ycomp_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
# 2.6 完全准确预测，补偿偏高：
y_s2_c6 = y_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]
yhat_s2_c6 = yhat_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c6 = ycomp_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]


# 对于补偿无效果这一大类中的几小类情况：



In [14]:
m11, m12 = len(y_s1_c1), len(y_s1_c2)  # 同侧补偿且变好
m13, m14 = len(y_s1_c3), len(y_s1_c4)  # 对侧补偿且变好
m15, m16 = len(y_s1_c5), len(y_s1_c6)  # 完全补偿
m1syn, m1opp, m1com = m11+m12, m13+m14, m15+m16  # 补偿变好时同侧、对侧、完全补偿的单品数
if m11+m12+m13+m14+m15+m16 == m1:
    print('计算正确，当补偿变好时，其下6种子情况的单品总数相加与补偿变好的单品总数相等：\n')
    print('预测销量与补偿后销量均比真实销量低，补偿变好且发生在同侧，该种情况有：{:.0f}个单品，\
占补偿变好单品数的比例为：{:.2f}%'.format(m11, m11/m1*100))
    print('预测销量与补偿后销量均比真实销量高，补偿变好且发生在同侧，该种情况有：{:.0f}个单品，\
占补偿变好单品数的比例为：{:.2f}%'.format(m12, m12/m1*100))
    print('预测销量偏低，补偿后销量偏高，补偿变好且发生在对侧，该种情况有：{:.0f}个单品，\
占补偿变好单品数的比例为：{:.2f}%'.format(m13, m13/m1*100))
    print('预测销量偏高，补偿后销量偏低，补偿变好且发生在对侧，该种情况有：{:.0f}个单品，\
占补偿变好单品数的比例为：{:.2f}%'.format(m14, m14/m1*100))
    print('预测销量偏低，完全补偿，该种情况有：{:.0f}个单品，占补偿变好单品数的比例为：{:.2f}%'.format(m15, m15/m1*100))
    print('预测销量偏高，完全补偿，该种情况有：{:.0f}个单品，占补偿变好单品数的比例为：{:.2f}%'.format(m16, m16/m1*100))
    print('补偿变好且发生在同侧的单品数有：{:.0f}个，占补偿变好单品数的比例为：{:.2f}%'.format(m1syn, m1syn/m1*100))
    print('补偿变好且发生在对侧的单品数有：{:.0f}个，占补偿变好单品数的比例为：{:.2f}%'.format(m1opp, m1opp/m1*100))
    print('完全补偿的单品数有：{:.0f}个，占补偿变好单品数的比例为：{:.2f}%'.format(m1com, m1com/m1*100))  
else: print('计算有误，当补偿变好时，其下6种子情况的单品总数相加与补偿变好的单品总数不等。')

计算正确，当补偿变好时，其下6种子情况的单品总数相加与补偿变好的单品总数相等：

预测销量与补偿后销量均比真实销量低，补偿变好且发生在同侧，该种情况有：571个单品，占补偿变好单品数的比例为：22.87%
预测销量与补偿后销量均比真实销量高，补偿变好且发生在同侧，该种情况有：603个单品，占补偿变好单品数的比例为：24.15%
预测销量偏低，补偿后销量偏高，补偿变好且发生在对侧，该种情况有：624个单品，占补偿变好单品数的比例为：24.99%
预测销量偏高，补偿后销量偏低，补偿变好且发生在对侧，该种情况有：646个单品，占补偿变好单品数的比例为：25.87%
预测销量偏低，完全补偿，该种情况有：30个单品，占补偿变好单品数的比例为：1.20%
预测销量偏高，完全补偿，该种情况有：23个单品，占补偿变好单品数的比例为：0.92%
补偿变好且发生在同侧的单品数有：1174个，占补偿变好单品数的比例为：47.02%
补偿变好且发生在对侧的单品数有：1270个，占补偿变好单品数的比例为：50.86%
完全补偿的单品数有：53个，占补偿变好单品数的比例为：2.12%


In [37]:
# 1.1 预测补偿均低，补偿同侧变好：

pva_s1_c1 = abs(y_s1_c1 - yhat_s1_c1)  # 预测偏差量
pva_s1_c1_sts = pva_s1_c1.describe()
pvr_s1_c1 = abs(pva_s1_c1[y_s1_c1!=0] / y_s1_c1[y_s1_c1!=0])*100  # 预测偏差量与真实销量百分比，在此时的条件下真实值(分母)为0的单品不能参与该指标计算，否则会使下面describe的统计指标失真。所以pvr_s1_c1计算的是真实销量不为0的那些单品。
pvr_s1_c1_sts = pvr_s1_c1.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c1_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c1_sts)
print('----------------------------------------')

cva_s1_c1 = abs(y_s1_c1 - ycomp_s1_c1)  # 补偿后剩余偏差量
cva_s1_c1_sts = cva_s1_c1.describe()
cvr_s1_c1 = abs(cva_s1_c1[y_s1_c1!=0] / y_s1_c1[y_s1_c1!=0])*100  # 补偿后剩余偏差量与真实销量百分比，在此时的条件下真实值(分母)为0的单品不能参与该指标计算，否则会使下面describe的统计指标失真。所以cvr_s1_c1计算的是真实销量不为0的那些单品。
cvr_s1_c1_sts = cvr_s1_c1.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c1_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c1_sts)
print('----------------------------------------')

ca_s1_c1 = abs(yhat_s1_c1 - ycomp_s1_c1)  # 补偿量
ca_s1_c1_sts = ca_s1_c1.describe()
car_s1_c1 = abs(ca_s1_c1[pva_s1_c1!=0] / pva_s1_c1[pva_s1_c1!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c1_sts = car_s1_c1.describe()
print('补偿量的统计指标：')
print(ca_s1_c1_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c1_sts)


预测偏差量的统计指标：
count    571.000000
mean       0.833305
std        0.200038
min        0.114785
25%        0.756802
50%        0.923458
75%        0.988032
max        0.999990
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    538.000000
mean      15.327069
std       21.343968
min        0.310229
25%        3.490284
50%        6.628755
75%       16.653447
max       99.999021
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    571.000000
mean       0.483068
std        0.292851
min        0.008851
25%        0.245252
50%        0.444113
75%        0.750987
max        0.999755
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    538.000000
mean       8.936756
std       15.093670
min        0.017356
25%        1.354168
50%        3.507878
75%        8.561540
max       95.892427
dtype: float64
----------------------------------------
补偿量的统计指标：
count    571.000000
mean       0.350237
std        0.271890
min        0.000235
25%        0.111682
50%        0.282138
75%        0.54931

In [7]:
m21, m22 = len(y_s2_c1), len(y_s2_c2)  # 同侧补偿且变差
m23, m24 = len(y_s2_c3), len(y_s2_c4)  # 对侧补偿且变差
m25, m26 = len(y_s2_c5), len(y_s2_c6)  # 完全准确预测
m2syn, m2opp, m2com = m21+m22, m23+m14, m25+m26  # 补偿变差时同侧、对侧、完全准确预测的单品数
if m21+m22+m23+m24+m25+m26 == m2:
    print('计算正确，当补偿变差时，其下6种子情况的单品总数相加与补偿变差的单品总数相等：\n')
    print('预测销量与补偿后销量均比真实销量低，补偿变差且发生在同侧，该种情况有：{:.0f}个单品，\
占补偿变差单品数的比例为：{:.2f}%'.format(m21, m21/m2*100))
    print('预测销量与补偿后销量均比真实销量高，补偿变差且发生在同侧，该种情况有：{:.0f}个单品，\
占补偿变差单品数的比例为：{:.2f}%'.format(m22, m22/m2*100))
    print('预测销量偏低，补偿后销量偏高，补偿变差且发生在对侧，该种情况有：{:.0f}个单品，\
占补偿变差单品数的比例为：{:.2f}%'.format(m23, m23/m2*100))
    print('预测销量偏高，补偿后销量偏低，补偿变差且发生在对侧，该种情况有：{:.0f}个单品，\
占补偿变差单品数的比例为：{:.2f}%'.format(m24, m24/m2*100))
    print('完全准确预测，补偿后销量偏低，该种情况有：{:.0f}个单品，占补偿变差单品数的比例为：{:.2f}%'.format(m25, m25/m2*100))
    print('完全准确预测，补偿后销量偏高，该种情况有：{:.0f}个单品，占补偿变差单品数的比例为：{:.2f}%'.format(m26, m26/m2*100))
    print('补偿变差且发生在同侧的单品数有：{:.0f}个，占补偿变差单品数的比例为：{:.2f}%'.format(m2syn, m2syn/m2*100))
    print('补偿变差且发生在对侧的单品数有：{:.0f}个，占补偿变差单品数的比例为：{:.2f}%'.format(m2opp, m2opp/m2*100))
    print('完全准确预测的单品数有：{:.0f}个，占补偿变差单品数的比例为：{:.2f}%'.format(m2com, m2com/m2*100))  
else: print('计算有误，当补偿变差时，其下6种子情况的单品总数相加与补偿变差的单品总数不等。')

计算正确，当补偿变差时，其下6种子情况的单品总数相加与补偿变差的单品总数相等：

预测销量与补偿后销量均比真实销量低，补偿变差且发生在同侧，该种情况有：590个单品，占补偿变差单品数的比例为：23.94%
预测销量与补偿后销量均比真实销量高，补偿变差且发生在同侧，该种情况有：636个单品，占补偿变差单品数的比例为：25.81%
预测销量偏低，补偿后销量偏高，补偿变差且发生在对侧，该种情况有：611个单品，占补偿变差单品数的比例为：24.80%
预测销量偏高，补偿后销量偏低，补偿变差且发生在对侧，该种情况有：569个单品，占补偿变差单品数的比例为：23.09%
完全准确预测，补偿后销量偏低，该种情况有：18个单品，占补偿变差单品数的比例为：0.73%
完全准确预测，补偿后销量偏高，该种情况有：40个单品，占补偿变差单品数的比例为：1.62%
补偿变差且发生在同侧的单品数有：1226个，占补偿变差单品数的比例为：49.76%
补偿变差且发生在对侧的单品数有：1257个，占补偿变差单品数的比例为：51.01%
完全准确预测的单品数有：58个，占补偿变差单品数的比例为：2.35%
